<a href="https://colab.research.google.com/github/MDMASIUD786/GithubOnlinePractise/blob/main/Copy_of_facialrecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git


Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 28.98 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [4]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
      key=row[2]
      if key in data:
        data[key].append(row[1])
      else:
        data[key]=[row[1]]

In [34]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt',
 'NEUTRAL',
 'SADNESS',
 'DISGUST',
 'FEAR',
 'SURPRISE',
 'ANGER',
 'HAPPINESS']

In [35]:

import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))



FileExistsError: ignored

In [36]:
from shutil import copyfile
split_size=0.8

for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]

  for image in train_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/testing',emotion,image)
    copyfile(source,dest)


In [37]:
#facial_recogn -3
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten


In [53]:
model=tf.keras.models.Sequential(
    [
     Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
     MaxPooling2D(2,2),
     Conv2D(32,(3,3),activation='relu'),
     MaxPooling2D(2,2),
     Conv2D(64,(3,3),activation='relu'),
     MaxPooling2D(2,2),
     Flatten(),
     Dense(1024,activation='relu'),
     Dense(15,activation='softmax')
    ]
)
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 6400)             

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [54]:
train_dir='/content/master_data/training'
test_dir='/content/master_data/testing'
train_datagen=ImageDataGenerator(rescale=1.0/255)

train_generator=train_datagen.flow_from_directory(
                                         train_dir,
                                         target_size=(100,100)
                                         ,class_mode='categorical',
                                         batch_size=128
)
test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(
                                         test_dir,
                                         target_size=(100,100)
                                         ,class_mode='categorical',
                                         batch_size=128
)

Found 10939 images belonging to 15 classes.
Found 2744 images belonging to 15 classes.


In [55]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01) #min_delta is 1% tbhi age jega otherwise stop

In [ ]:
model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
86/86 [==============================] - 19s 217ms/step - loss: 2.1339 - acc: 0.4669 - val_loss: 1.0363 - val_acc: 0.5463
Epoch 2/10
86/86 [==============================] - 18s 211ms/step - loss: 1.0849 - acc: 0.5350 - val_loss: 1.0420 - val_acc: 0.5044
Epoch 3/10
86/86 [==============================] - 18s 210ms/step - loss: 0.9835 - acc: 0.6023 - val_loss: 0.9515 - val_acc: 0.6119
Epoch 4/10
 1/86 [..............................] - ETA: 16s - loss: 0.8816 - acc: 0.6172